In [1]:
!pip install pybullet imageio-ffmpeg

import os
import time
import math
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import pybullet as p
import pybullet_data
import cv2
import imageio_ffmpeg
from base64 import b64encode
from IPython.display import HTML

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 MB 4.6 MB/s eta 0:00:00


In [2]:
!git clone https://github.com/Gaianeve/Real_Mega_Fufi.git
%cd Real_Mega_Fufi/Robot/1LegTrail/

Cloning into 'Real_Mega_Fufi'...
remote: Enumerating objects: 1480, done.
remote: Counting objects: 100% (373/373), done.
remote: Compressing objects: 100% (171/171), done.
remote: Total 1480 (delta 351), reused 202 (delta 202), pack-reused 1107 (from 1)
Receiving objects: 100% (1480/1480), 24.01 MiB | 14.88 MiB/s, done.
Resolving deltas: 100% (981/981), done.
/content/Real_Mega_Fufi/Robot/1LegTrail


In [3]:
import numpy as np
import imageio_ffmpeg
import pybullet as p
import math

# Configura la simulazione e la fotocamera
p.connect(p.DIRECT)  # Usa la modalità senza GUI per evitare un impatto sulle prestazioni
p.resetSimulation() # Lui va chiamato la mattina appena apri il computer
p.setGravity(0, 0, -9.81)
# Aggiungi il percorso dei dati PyBullet
p.setAdditionalSearchPath(pybullet_data.getDataPath())

In [10]:
plane_pos = [0,0,0]
plane = p.loadURDF("plane.urdf", plane_pos, useFixedBase=True)

# position of the base of the model
PIP_position = [0,0,0]
orientation  = [0,0,0]
PIP_orientation = p.getQuaternionFromEuler(orientation)
PIP_id = p.loadURDF("oneleg.urdf", basePosition = PIP_position, baseOrientation = PIP_orientation, \
                      useFixedBase=False)

#rimettiamo il centro di mass aper sicurezza
p.resetBasePositionAndOrientation(PIP_id, [0, 0, 0.1], [0, 0, 0, 1])



In [11]:
#try setting il motore fisico
p.setPhysicsEngineParameter(
    fixedTimeStep=1/240,
    numSolverIterations=100,
    contactERP=0.2
)


In [12]:
#change colors
p.changeVisualShape(PIP_id, -1, rgbaColor=[1, 0.65, 0, 1]) #terreno arancione
p.changeVisualShape(PIP_id, 0, rgbaColor=[0.5, 1, 0.5, 1]) #colonna verdino
p.changeVisualShape(PIP_id, 1, rgbaColor=[0.5, 0.8, 1, 1]) #colonna azzurrino

In [13]:
link_name_to_index = {p.getBodyInfo(PIP_id)[0].decode('UTF-8'):-1,}

for id in range(p.getNumJoints(PIP_id)):
  name = p.getJointInfo(PIP_id, id)[12].decode('UTF-8')
  link_name_to_index[name] = id

#proviamo a mettere dell'attrito per farlo stare fermo
p.changeDynamics(PIP_id, -1, linearDamping=0.04, angularDamping=0.01, restitution = 0)
p.changeDynamics(PIP_id, -1, lateralFriction=0.8, spinningFriction=0.3, rollingFriction=0.1)

In [14]:
#get initial position (rest)
column_position, _ = p.getBasePositionAndOrientation(PIP_id)
#externale force due to seismic motion
force_position = column_position

#joint initial orientation and coordinate
joint_position, orientation = p.getLinkState(PIP_id, 0)[0:2]
joint_orientation = p.getEulerFromQuaternion(orientation)
theta_x_rest_joint = joint_orientation[0]
theta_y_rest_joint = joint_orientation[1]


#elastic force:
def spring(theta_x_now, theta_y_now, theta_x_rest, theta_y_rest):
  force_module_x = -1800*0.08*(theta_x_now - theta_x_rest)
  force_module_y =  -1800*0.08*(theta_y_now - theta_y_rest)
  return force_module_x, force_module_y

In [15]:

# Parametri della fotocamera
cam_target_pos = [0, 0, 0]
cam_distance = 2
cam_yaw, cam_pitch, cam_roll = 50, -30.0, 0
cam_width, cam_height = 480, 360
cam_up_axis_idx = 2
cam_fov = 60
cam_near_plane, cam_far_plane = 0.01, 100

# Imposta la registrazione del video
vid = imageio_ffmpeg.write_frames('simulation_output.mp4', (cam_width, cam_height), fps=30)
vid.send(None)  # Inizializza con un fotogramma vuoto
p.stepSimulation()  # Avanza di un passo nella simulazione

()

In [16]:
# Liste per registrare le posizioni
leg_positions_x, leg_positions_y, leg_position_z = [], [], []
joint_positions_x, joint_positions_y, joint_position_z = [], [], []
column_positions_x, column_positions_y, column_positions_z = [], [], []

#liste per la variazione di posizione
delta_x_leg, delta_y_leg = [],[]
delta_x_joint, delta_y_joint = [],[]
delta_x_column, delta_y_column = [],[]

#lista per le forze
force_x_column, force_y_column = [],[]
force_x_leg, force_y_leg = [],[]
force_x_joint, force_y_joint = [],[]
theta_joint_x, theta_joint_y = [],[]
force_module_botta = []

In [ ]:
# Ciclo principale della simulazione
# Loop di simulazione
for step in range(50000):
  # Determinare la forza in base allo step
  if  220 <= step < 460:
      force_module = np.array([4e-8, 0, 0])  # Nessuna forza
  elif 462 <= step < 702:
      force_module = np.array([-4e-8, 0, 0])  # Forza negativa
  #elif 6 <= step < 7:
      #force_module = np.array([0, 0, 0])  # Nessuna forza
  #elif 7 <= step < 8:
      #force_module = np.array([0.000005, 0, 0])  # Forza positiva
  else:
      force_module = np.array([0, 0, 0])  # Nessuna forza
  # Applicare la forza al sistema
  p.applyExternalForce(
      objectUniqueId=PIP_id,
      linkIndex=-1,
      forceObj=force_module,
      posObj=force_position,
      flags=p.WORLD_FRAME
  )


  # Salva il modulo della forza per il grafico
  force_module_botta.append(force_module[0])

  #elastic force on the joint
  elastic_force_x_joint, elastic_force_y_joint = spring(joint_orientation[0], \
                                                        joint_orientation[1], \
                                                        theta_x_rest_joint, \
                                                        theta_y_rest_joint )

  elastic_force_joint = [elastic_force_x_joint, elastic_force_y_joint,0]
  force_x_joint.append(elastic_force_x_joint)
  force_y_joint.append(elastic_force_y_joint)

  #elastic force on joint
  p.applyExternalForce(
      objectUniqueId= PIP_id,
      linkIndex=0,
      forceObj=elastic_force_joint,
      posObj=joint_position,
      flags=p.WORLD_FRAME
    )

  #get new positions
  leg_position, _ = p.getLinkState(PIP_id,1)[0:2]
  leg_positions_x.append(leg_position[0])
  leg_positions_y.append(leg_position[1])
  leg_position_z.append(leg_position[2])

  #column position
  column_position, _ = p.getBasePositionAndOrientation(PIP_id)
  column_positions_x.append(column_position[0])
  column_positions_y.append(column_position[1])
  column_positions_z.append(column_position[2])

  #joint position
  joint_position, orientation = p.getLinkState(PIP_id,0)[0:2]
  joint_positions_x.append(joint_position[0])
  joint_positions_y.append(joint_position[1])
  joint_position_z.append(joint_position[2])

  joint_orientation = p.getEulerFromQuaternion(orientation)
  theta_joint_x.append(joint_orientation[0])
  theta_joint_y.append(joint_orientation[1])

  # Aggiorna il video
  # Calcola la vista e la proiezione della fotocamera
  cam_view_matrix = p.computeViewMatrixFromYawPitchRoll(
      cam_target_pos, cam_distance, cam_yaw, cam_pitch, cam_roll, cam_up_axis_idx
  )
  cam_projection_matrix = p.computeProjectionMatrixFOV(
      cam_fov, cam_width / cam_height, cam_near_plane, cam_far_plane
  )

  # Cattura l'immagine
  image = p.getCameraImage(cam_width, cam_height, cam_view_matrix, cam_projection_matrix)[2][:, :, :3]

  # Scrivi il fotogramma nel video
  vid.send(np.ascontiguousarray(image))

  p.stepSimulation()  # Avanza di un passo nella simulazione

# Rilascia le risorse video
vid.close()
# Disconnessione dalla simulazione
p.disconnect()

In [ ]:
# Ora visualizziamo il video registrato nel notebook
mp4 = open('simulation_output.mp4', 'rb').read()  # Leggi il file video
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()  # Convertilo in Base64

# Visualizza il video nel notebook Jupyter
HTML('<video width=480 controls><source src="%s" type="video/mp4"></video>' % data_url)

In [ ]:
##plotting positions
plt.figure('Leg positions',  figsize = (12,7))
plt.plot(leg_positions_x, label='x')
plt.plot(leg_positions_y, label='y')
plt.xlabel('Simulation step')
plt.ylabel('Leg Position [m]')
plt.legend()
plt.grid()
plt.minorticks_on()
#plt.ylim(leg_positions_x[0],0.6e-13)
plt.xlim(0,50000)
#print(leg_positions_x[0])
plt.show()

In [ ]:
##plotting positions
plt.figure('Leg positions',  figsize = (15,7))
plt.plot(leg_position_z[100:340], label='z')
plt.xlabel('Simulation step')
plt.ylabel('Leg Position [m]')
plt.legend()
plt.grid()
plt.minorticks_on()
#plt.xlim(0,240)
min_plot = min(leg_position_z[100:340]) - 0.9e-14
max_plot = max(leg_position_z[100:340]) + 0.9e-14
#plt.ylim(0.6184900000818735,0.6184900000818800)
plt.show()

In [ ]:
print(min(leg_position_z[100:340]))
print(max(leg_position_z[100:340]))

In [ ]:

#plot column positions
plt.figure('column', figsize = (12,7))
#plt.plot(column_positions_x, label='x')
#plt.plot( column_positions_y, label='y')
plt.plot(column_positions_z[100:340], label='z')
plt.xlabel('Simulation step')
plt.ylabel('column Position [m]')
plt.legend()
plt.grid()
plt.minorticks_on()
#plt.xlim(0,240)
min_plot = min(column_positions_z[100:340]) - 1e-16
max_plot = max(column_positions_z[100:340]) + 1e-16
#plt.ylim(min_plot, max_plot)
plt.show()

In [ ]:
#proviamo a verificare che stia fermo a botte di 240
print(min(column_positions_z[100:340]))
print(max(column_positions_z[100:340]))

In [ ]:
#plot elastic forces applied
plt.figure('Theta joint', figsize = (12,7))
plt.plot(theta_joint_y, label='y')
plt.xlabel('Simulation step')
plt.ylabel('Joint angular postion [rad]')
plt.legend()
plt.grid()
#plt.ylim(theta_joint_y[0],9e-14)
#plt.xlim(0,10000)
plt.minorticks_on()

In [ ]:
#plot elastic forces applied
plt.figure('Theta joint', figsize=(12,7))
plt.plot(theta_joint_x, label='x')
plt.xlabel('Simulation step')
plt.ylabel('Joint angular postion [rad]')
plt.legend()
plt.grid()
#plt.ylim(-1e-15,3e-15)
#plt.xlim(300,400)
plt.minorticks_on()

In [ ]:
print(theta_joint_y[1])
print(theta_joint_y[0])

In [ ]:
plt.figure('elastic force joint')
plt.plot(force_x_joint, label='x')
#plt.plot(force_y_joint, label='y')
plt.xlabel('Simulation step')
plt.ylabel('elastic force on the joint [N]')
plt.legend()
plt.grid()
plt.minorticks_on()
#plt.xlim(50,1500)

In [ ]:
plt.figure('force module')
plt.plot(force_module_botta, label='x')
plt.xlabel('Simulation step')
plt.ylabel('force module [N]')
plt.legend()
plt.grid()
plt.minorticks_on()
